Plano de negócio 

- Saída: 
    - Uma tabela com as informações dos l




- Processo: A sequência de passos organizada pela lógica de execução.
    - Analisar o HTML da página
    - Pesquisar melhor forma de realizar a extração de dados
    - Coleta os dados seguintes dados categoria| nome_livro|avaliação_consumidor|estoque|preço
    - Limpeza dos dados


- Entrada:

     1. Fonte de dados

    - Site da Book to Scrape: https://books.toscrape.com
    

     2. Ferramentas

    - Python 3.8.0
    - Bibliotecas de Webscrapping (BS4, Selenium)
    - Jupyter Notebooks (Análises e Prototipagem)
    


# 0.0 Imports

In [2]:
import re
import requests
import pandas as pd
import numpy as np
import seaborn as sns


from datetime import datetime
from bs4 import BeautifulSoup

from IPython.core.display  import HTML
from IPython.display       import Image

from matplotlib            import pyplot as plt

# 0.1 Helpe Functions

In [3]:
def jupyter_settings():
    
    %matplotlib inline 
    %pylab inline
    
    plt.style.use('bmh')
    plt.rcParams['figure.figsize']=[20,10]
    plt.rcParams['font.size']=10
    
    display( HTML('<style>.container {width:100% !important; }</style>'))
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    #pd.set_options('display.expand_frame_repr',False )
    
    sns.set()

In [4]:
jupyter_settings()

warnings.filterwarnings ('ignore')

Populating the interactive namespace from numpy and matplotlib


/home/bruna/.pyenv/versions/3.8.0/envs/datascienceemproducao/lib/python3.8/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


NameError: name 'warnings' is not defined

# 1.0 Data Collect

In [ ]:
book_title = []
book_price = []
book_stock = []
book_rating = []
book_category = []
quantidade = 0


for page in range(1,51):
    
    #Get webpage data
    root_url = 'https://books.toscrape.com/catalogue/'
    url = 'https://books.toscrape.com/catalogue/page-{}.html'.format(page)
    
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5),AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    
    response = requests.get(url,headers=headers)
    
    # Make Soup
    soup = BeautifulSoup(response.text,'html.parser') 
    
    
    
    book = soup.find_all('article', class_='product_pod')
    
    for book in book:
        
        # Get Hyper Link
        base_url = book.h3.a['href'].strip()
        url = root_url + base_url
        
        page_single = requests.get(url)
        soup_single = BeautifulSoup(page_single.text,'html.parser') 
             
        
        book_single = soup_single.find('article', class_='product_page')

        info_book = book_single.find('div', class_='product_main')


        #Scraping Data
        title = info_book.h1.get_text().strip()
        price = info_book.find('p', class_='price_color').get_text()
        stock = info_book.find('p', class_='availability').get_text().split()
        rating = info_book.find('p','star-rating')['class'][1].strip()
        category = soup_single.find('ul', class_='breadcrumb').find_all('li')[2].a.get_text() 

        #clean data
        stock_clean = re.findall(r"\d+", stock)[0]

        #rating_clean = clean.str_to_int(rating)

        book_title.append(title)
        book_price.append(price)
        book_stock.append(stock_clean)
        book_rating.append(rating)
        book_category.append(category)


        quantidade = quantidade +1
        print(quantidade)

In [ ]:
data = pd.DataFrame({'book_category': book_category,
                    'book_title': book_title,
                    'book_price': book_price,
                    'book_stock': book_stock,
                    'book_rating': book_rating})
                    


In [ ]:
data = data.loc[((data['book_category'] == 'Classics') | (data['book_category'] == 'Science Fiction')
                |(data['book_category'] == 'Humor')|(data['book_category'] == 'Business'))]



In [6]:
from datetime import datetime
#scrapy datetime
data.insert(1, 'scrapy_datetime',(datetime.now().strftime('%Y-%m-%d %H:%M:%S')),allow_duplicates=False)

In [7]:
data.head()

,book_category,scrapy_datetime,book_title,book_price,book_stock,book_rating
0,Business,2021-05-25 20:32:44,The Dirty Little Secrets of Getting Your Dream...,Â£33.34,"['In', 'stock', '(19', 'available)']",Four
1,Science Fiction,2021-05-25 20:32:44,Mesaerion: The Best Science Fiction Stories 18...,Â£37.59,"['In', 'stock', '(19', 'available)']",One
2,Science Fiction,2021-05-25 20:32:44,Join,Â£35.67,"['In', 'stock', '(16', 'available)']",Five
3,Science Fiction,2021-05-25 20:32:44,"William Shakespeare's Star Wars: Verily, A New...",Â£43.30,"['In', 'stock', '(15', 'available)']",Four
4,Business,2021-05-25 20:32:44,The Third Wave: An Entrepreneurâs Vision of ...,Â£12.61,"['In', 'stock', '(15', 'available)']",Five


In [8]:
data.to_csv('/home/bruna/Documentos/new_project/dataset_v1.csv',index=False)

# 2.0 Clean Data

In [5]:
data= pd.read_csv('/home/bruna/Documentos/new_project/data/dataset_v1.csv')

In [ ]:
## book category
data['book_category'] = data['book_category'].apply(lambda x: x.lower()) 

# product price 
data['book_price']= data['book_price'].apply(lambda x: x.replace('Â£','') if pd.notnull(x) else x).astype(float)

# book rating
data['book_rating'] = data['book_rating'].apply(lambda x: x.lower()) 

# book stock

regex ='\W((.+?),(.+?)),'

data['book_stock']= data['book_stock'].apply(lambda x: re.match(regex, x).group(1))
data['book_stock']= data['book_stock'].apply(lambda x: x.strip("'"))
data['book_stock']= data['book_stock'].apply(lambda x: x.replace("'" , "").replace("," , ""))


data.sample(5)

In [9]:
data.sample(5)

,book_category,scrapy_datetime,book_title,book_price,book_stock,book_rating
35,Classics,2021-05-25 20:32:44,The Picture of Dorian Gray,Â£29.70,"['In', 'stock', '(3', 'available)']",Two
2,Science Fiction,2021-05-25 20:32:44,Join,Â£35.67,"['In', 'stock', '(16', 'available)']",Five
4,Business,2021-05-25 20:32:44,The Third Wave: An Entrepreneurâs Vision of ...,Â£12.61,"['In', 'stock', '(15', 'available)']",Five
37,Classics,2021-05-25 20:32:44,The Complete Stories and Poems (The Works of E...,Â£26.78,"['In', 'stock', '(3', 'available)']",Four
5,Science Fiction,2021-05-25 20:32:44,The Project,Â£10.65,"['In', 'stock', '(15', 'available)']",One


In [11]:
data.to_csv('/home/bruna/Documentos/new_project/dataset_v2.csv',index=False)